In [1]:
import torch
from train.train_detection_networks import train
# from models.detection.ofa_mbv3_w12_fcos import get_ofa_mbv3_w12_fcos_model, load_pretrained_fcos, set_training_params
from models.detection.ofa_mbv3_w12_fasterrcnn import get_ofa_mbv3_w12_fasterrcnn_model, load_pretrained_fasterrcnn, set_training_params
# model = get_ofa_mbv3_w12_fcos_model()
model = get_ofa_mbv3_w12_fasterrcnn_model()
set_training_params(model, is_backbone_body_need_training=False)



Using cache found in C:\Users\wydai/.cache\torch\hub\mit-han-lab_once-for-all_master


2024-12-10 19:38:07,094 - model - INFO - backbone.fpn.inner_blocks.0.0.weight
2024-12-10 19:38:07,096 - model - INFO - backbone.fpn.inner_blocks.0.0.bias
2024-12-10 19:38:07,098 - model - INFO - backbone.fpn.inner_blocks.1.0.weight
2024-12-10 19:38:07,100 - model - INFO - backbone.fpn.inner_blocks.1.0.bias
2024-12-10 19:38:07,101 - model - INFO - backbone.fpn.inner_blocks.2.0.weight
2024-12-10 19:38:07,102 - model - INFO - backbone.fpn.inner_blocks.2.0.bias
2024-12-10 19:38:07,103 - model - INFO - backbone.fpn.inner_blocks.3.0.weight
2024-12-10 19:38:07,105 - model - INFO - backbone.fpn.inner_blocks.3.0.bias
2024-12-10 19:38:07,106 - model - INFO - backbone.fpn.layer_blocks.0.0.weight
2024-12-10 19:38:07,107 - model - INFO - backbone.fpn.layer_blocks.0.0.bias
2024-12-10 19:38:07,108 - model - INFO - backbone.fpn.layer_blocks.1.0.weight
2024-12-10 19:38:07,109 - model - INFO - backbone.fpn.layer_blocks.1.0.bias
2024-12-10 19:38:07,111 - model - INFO - backbone.fpn.layer_blocks.2.0.weigh

In [2]:
def hook_fn_fasterrcnn(module, input, output):
    # 打印output字典中的key
    print(output.keys())
    for key in output.keys():
        print(key)
        print(output[key].shape)
    return output

In [3]:
images, boxes = torch.rand(4, 3, 300, 600), torch.rand(4, 11, 4)
boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
labels = torch.randint(1, 91, (4, 11))
images = list(image for image in images)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    d['labels'] = labels[i]
    targets.append(d)
model.to('cpu')
# model.head.register_forward_hook(hook_fn)
hook = model.backbone.fpn.register_forward_hook(hook_fn_fasterrcnn)
output = model(images, targets)

odict_keys(['0', '1', '2', '3', 'pool'])
0
torch.Size([4, 256, 84, 168])
1
torch.Size([4, 256, 42, 84])
2
torch.Size([4, 256, 42, 84])
3
torch.Size([4, 256, 21, 42])
pool
torch.Size([4, 256, 11, 21])


In [ ]:


# 用完后记得移除
hook.remove()